In [1]:
!git clone https://github.com/kcnishan/Nepali_handwritten_digits_recognition

Cloning into 'Nepali_handwritten_digits_recognition'...
remote: Enumerating objects: 92087, done.
remote: Total 92087 (delta 0), reused 0 (delta 0), pack-reused 92087
Receiving objects: 100% (92087/92087), 67.09 MiB | 26.41 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (92306/92306), done.


In [0]:
import os
base_dir = 'Nepali_handwritten_digits_recognition/dataset/'
train_dir = os.path.join(base_dir, 'Train')
test_dir = os.path.join(base_dir, 'Test')
train_consonant_dir = os.path.join(train_dir, 'consonants/')
train_digits_dir = os.path.join(train_dir, 'digits/')
test_consonant_dir = os.path.join(test_dir, 'consonants/')
test_digits_dir = os.path.join(test_dir, 'digits/')

In [0]:
import shutil
import tensorflow as tf
class customCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if(logs.get('acc')>0.99):
      print("99 percent accuracy reached, so stopping")
      self.model.stop_training = True
mycallback = customCallback()

In [4]:
for direct in os.listdir(train_digits_dir):
  print(direct)
  prev_data_path = os.path.join(train_digits_dir, direct)
  new_data_path = os.path.join(train_consonant_dir, direct)
  shutil.copytree(prev_data_path, new_data_path)

digit_0
digit_4
digit_8
digit_9
digit_6
digit_5
digit_2
digit_3
digit_1
digit_7


In [5]:
os.listdir(train_consonant_dir)

['character_12_thaa',
 'character_22_pha',
 'character_28_la',
 'character_16_tabala',
 'character_36_gya',
 'character_4_gha',
 'character_8_ja',
 'character_25_ma',
 'character_2_kha',
 'character_10_yna',
 'character_14_dhaa',
 'digit_0',
 'character_33_ha',
 'character_11_taamatar',
 'character_29_waw',
 'digit_4',
 'character_30_motosaw',
 'character_26_yaw',
 'character_27_ra',
 'character_9_jha',
 'character_34_chhya',
 'character_19_dha',
 'character_3_ga',
 'digit_8',
 'character_23_ba',
 'digit_9',
 'digit_6',
 'character_7_chha',
 'character_21_pa',
 'digit_5',
 'digit_2',
 'character_31_petchiryakha',
 'character_20_na',
 'character_18_da',
 'character_35_tra',
 'digit_3',
 'character_13_daa',
 'character_17_tha',
 'character_32_patalosaw',
 'character_1_ka',
 'digit_1',
 'digit_7',
 'character_24_bha',
 'character_6_cha',
 'character_5_kna',
 'character_15_adna']

In [6]:
direct = " "
for direct in os.listdir(test_digits_dir):
  print(direct)
  prev_data_path = os.path.join(test_digits_dir, direct)
  new_data_path = os.path.join(test_consonant_dir, direct)
  shutil.copytree(prev_data_path, new_data_path)

digit_0
digit_4
digit_8
digit_9
digit_6
digit_5
digit_2
digit_3
digit_1
digit_7


In [7]:
import cv2
image = cv2.imread(test_consonant_dir+'/'+'digit_0'+'/'+'103277.png')
print(image.shape)

(32, 32, 3)


In [8]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_consonant_dir, target_size = (32, 32), batch_size = 100, class_mode = 'categorical')

Using TensorFlow backend.


Found 78200 images belonging to 46 classes.


In [9]:
validation_generator = test_datagen.flow_from_directory(test_consonant_dir, target_size = (32, 32), batch_size = 100, class_mode = 'categorical')

Found 14100 images belonging to 46 classes.


In [10]:
#generator yields batches indefinitely: so we need to break the iteration loop at some point
for data_batch, labels_batch in train_generator:
    print('data_batch_shape:', data_batch.shape)
    print('labels_batch_shape:', labels_batch.shape)
    break

data_batch_shape: (100, 32, 32, 3)
labels_batch_shape: (100, 46)


In [0]:
str_labels = os.listdir(test_consonant_dir)

In [0]:
#building the neural newtork
from keras import layers
from keras import models
model = models.Sequential()

In [0]:
model.add(layers.Conv2D(64, (3,3), activation = 'relu', input_shape = (32, 32, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(46, activation = 'softmax'))

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1180160   
_________________________________________________________________
dense_2 (Dense)              (None, 46)               

In [0]:
from keras import optimizers
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

In [17]:
history = model.fit_generator(train_generator, steps_per_epoch = 20, epochs = 50, validation_data = validation_generator, validation_steps = 50, callbacks = [mycallback])

Epoch 1/50
20/20 [==============================] - 9s 425ms/step - loss: 3.2751 - acc: 0.2110 - val_loss: 2.1313 - val_acc: 0.3958
Epoch 2/50
20/20 [==============================] - 2s 87ms/step - loss: 1.7286 - acc: 0.5400 - val_loss: 1.5442 - val_acc: 0.6432
Epoch 3/50
20/20 [==============================] - 2s 92ms/step - loss: 1.1582 - acc: 0.6640 - val_loss: 0.9176 - val_acc: 0.7282
Epoch 4/50
20/20 [==============================] - 2s 89ms/step - loss: 0.8926 - acc: 0.7480 - val_loss: 0.8666 - val_acc: 0.7742
Epoch 5/50
20/20 [==============================] - 2s 87ms/step - loss: 0.7143 - acc: 0.8010 - val_loss: 1.0924 - val_acc: 0.7818
Epoch 6/50
20/20 [==============================] - 2s 90ms/step - loss: 0.5792 - acc: 0.8415 - val_loss: 0.6435 - val_acc: 0.8292
Epoch 7/50
20/20 [==============================] - 2s 87ms/step - loss: 0.5093 - acc: 0.8455 - val_loss: 0.7861 - val_acc: 0.8496
Epoch 8/50
20/20 [==============================] - 2s 87ms/step - loss: 0.5006 - 